## Data Collection

In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\harsu\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## Load the Dataset
data= gutenberg.raw('shakespeare-hamlet.txt')

## Save dataset to a file
with open('hamlet.txt', 'w') as file:
    file.write(data)

## Data Preprocessing

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [69]:
## Load Dataset
with open('hamlet.txt','r')as file:
    text= file.read(70000).lower() # Limiting data read to 70,000 characters due to memory read issue


## Tokenize the text
tokenizer= Tokenizer()
tokenizer.fit_on_texts([text])
total_words= len(tokenizer.word_index)+1
total_words

2712

In [70]:
# Create input sequences
input_sequence=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequence.append(n_gram_sequence)
input_sequence

[[1, 611],
 [1, 611, 4],
 [1, 611, 4, 52],
 [1, 611, 4, 52, 38],
 [1, 611, 4, 52, 38, 980],
 [1, 611, 4, 52, 38, 980, 981],
 [1, 611, 4, 52, 38, 980, 981, 982],
 [612, 983],
 [612, 983, 984],
 [612, 983, 984, 985],
 [70, 219],
 [70, 219, 2],
 [70, 219, 2, 613],
 [70, 219, 2, 613, 220],
 [70, 219, 2, 613, 220, 986],
 [219, 614],
 [219, 614, 48],
 [221, 162],
 [221, 162, 356],
 [221, 162, 356, 20],
 [221, 162, 356, 20, 276],
 [221, 162, 356, 20, 276, 436],
 [21, 60],
 [250, 222],
 [250, 222, 309],
 [250, 222, 309, 1],
 [250, 222, 309, 1, 39],
 [221, 219],
 [250, 25],
 [221, 7],
 [221, 7, 62],
 [221, 7, 62, 40],
 [221, 7, 62, 40, 987],
 [221, 7, 62, 40, 987, 90],
 [221, 7, 62, 40, 987, 90, 21],
 [221, 7, 62, 40, 987, 90, 21, 437],
 [250, 79],
 [250, 79, 69],
 [250, 79, 69, 988],
 [250, 79, 69, 988, 438],
 [250, 79, 69, 988, 438, 615],
 [250, 79, 69, 988, 438, 615, 71],
 [250, 79, 69, 988, 438, 615, 71, 3],
 [250, 79, 69, 988, 438, 615, 71, 3, 439],
 [250, 79, 69, 988, 438, 615, 71, 3, 439

In [71]:
## Pad Sequences
max_sequence_len= max([len(x) for x in input_sequence])
max_sequence_len

14

In [72]:
input_sequence= np.array(pad_sequences(input_sequence, maxlen=max_sequence_len, padding='pre'))
#input_sequence=input_sequence[:35000]
input_sequence

array([[   0,    0,    0, ...,    0,    1,  611],
       [   0,    0,    0, ...,    1,  611,    4],
       [   0,    0,    0, ...,  611,    4,   52],
       ...,
       [   0,    0,    0, ...,    0,   39,  387],
       [   0,    0,    0, ...,   39,  387,  673],
       [   0,    0,    0, ...,  387,  673, 2711]])

In [73]:
## Create predictors and labels
import tensorflow as tf

x,y= input_sequence[:, :-1], input_sequence[:,-1]

In [74]:
x

array([[  0,   0,   0, ...,   0,   0,   1],
       [  0,   0,   0, ...,   0,   1, 611],
       [  0,   0,   0, ...,   1, 611,   4],
       ...,
       [  0,   0,   0, ...,   0,   0,  39],
       [  0,   0,   0, ...,   0,  39, 387],
       [  0,   0,   0, ...,  39, 387, 673]])

In [75]:
y

array([ 611,    4,   52, ...,  387,  673, 2711])

In [76]:
y= tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [77]:
# Train test split
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.2)

## Train LSTM

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM, Dense, Dropout

## Define model
model= Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

## Compile the model
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 13, 100)           271200    
                                                                 
 lstm_2 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_1 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               100400    
                                                                 
 dense_1 (Dense)             (None, 2712)              273912    
                                                                 
Total params: 796112 (3.04 MB)
Trainable params: 796112 (3.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Train Model

In [83]:
history=model.fit(x_train,y_train,epochs=100, validation_data=(x_test,y_test), verbose=1)

Epoch 1/100
276/276 [==============================] - 6s 21ms/step - loss: 3.6367 - accuracy: 0.2313 - val_loss: 9.3513 - val_accuracy: 0.0498
Epoch 2/100
276/276 [==============================] - 7s 24ms/step - loss: 3.5775 - accuracy: 0.2362 - val_loss: 9.4267 - val_accuracy: 0.0534
Epoch 3/100
276/276 [==============================] - 6s 23ms/step - loss: 3.5236 - accuracy: 0.2481 - val_loss: 9.5278 - val_accuracy: 0.0530
Epoch 4/100
276/276 [==============================] - 6s 23ms/step - loss: 3.4674 - accuracy: 0.2528 - val_loss: 9.5972 - val_accuracy: 0.0507
Epoch 5/100
276/276 [==============================] - 7s 24ms/step - loss: 3.4129 - accuracy: 0.2655 - val_loss: 9.7210 - val_accuracy: 0.0539
Epoch 6/100
276/276 [==============================] - 6s 23ms/step - loss: 3.3663 - accuracy: 0.2676 - val_loss: 9.7843 - val_accuracy: 0.0489
Epoch 7/100
276/276 [==============================] - 7s 24ms/step - loss: 3.3123 - accuracy: 0.2832 - val_loss: 9.8626 - val_accuracy:

## Prediction Function

This function predicts the next word given an input text using a trained LSTM model.

### 🔍 How It Works:
1. **Tokenization**: Converts the input text into a sequence of integers.
2. **Trimming**: Ensures the sequence fits within the `max_sequence_len - 1` constraint by retaining only the most recent tokens if needed.
3. **Padding**: Pads the sequence from the left to match the input length expected by the model.
4. **Prediction**: Uses the model to predict the next word’s index.
5. **Decoding**: Finds the corresponding word from the tokenizer’s word index.

Returns the predicted word or `None` if not found.

In [ ]:
def predict_next_word(input_text, tokenizer, model, max_sequence_len):
    token_list= tokenizer.texts_to_sequences([input_text])[0]
    if len(token_list)>=max_sequence_len:
        token_list=token_list[-(max_sequence_len-1):]
    token_list= pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted= model.predict(token_list,verbose=0)
    predicted_word_index= np.argmax(predicted, axis=1)  # We extract the index of the highest probability word
    for word,index in tokenizer.word_index.items():
        if index== predicted_word_index:
            return word
    return None

In [93]:
input_text= " You are a"
max_sequence_len= model.input_shape[1]+1
print(f"Input text {input_text}")
next_word= predict_next_word(input_text, tokenizer, model, max_sequence_len )
print(f" Predicted word: {next_word}")

Input text  You are a
 Predicted word: catch
